In [7]:
import ee
import geemap
import geopandas as gpd
from ipyleaflet import GeoJSON
from zipfile import ZipFile

ee.Initialize()

initDate = '2018-05-31'
finalDate = '2022-12-31'
s5p_layer = 'CH4_column_volume_mixing_ratio_dry_air_bias_corrected'
s5p_methane_dataset = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CH4")
s5p_metane = s5p_methane_dataset.select(s5p_layer).filterDate(initDate, finalDate)


Map = geemap.Map()

Colombia = ee.FeatureCollection('users/hmojica/data/departamentos')
style = {'color': '000000', 'width': 2, 'lineType': 'solid', 'fillColor': '00000000'}
Map.addLayer(Colombia.style(**style), {}, "Departamentos")



departamentosNombres = Colombia.aggregate_array('DPTO_CNMBR').getInfo()


s5p_methane_viz = {
    'min': 1600,
    'max': 2000,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
    'opacity': 1.0
}

Map


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [8]:
# esto plotea la capa de metano departamento a departamento
"""
for dpmnto in departamentosNombres:
    departamento_shp = Colombia.filterMetadata('DPTO_CNMBR', 'equals', dpmnto)
    s5p_metane_dpto = s5p_metane.filterBounds(departamento_shp)
    s5p_metane_img = s5p_metane_dpto.map(lambda image: image.clip(departamento_shp))
    

    Map.addLayer(s5p_metane_img.mean(), s5p_methane_viz, 'methane in ' + dpmnto)
"""

"\nfor dpmnto in departamentosNombres:\n    departamento_shp = Colombia.filterMetadata('DPTO_CNMBR', 'equals', dpmnto)\n    s5p_metane_dpto = s5p_metane.filterBounds(departamento_shp)\n    s5p_metane_img = s5p_metane_dpto.map(lambda image: image.clip(departamento_shp))\n    \n\n    Map.addLayer(s5p_metane_img.mean(), s5p_methane_viz, 'methane in ' + dpmnto)\n"

In [11]:
# Aquí se plotea usando un shapefile del mundo y filtrando solo colombia

countries = ee.FeatureCollection('users/hmojica/data/countries')

colombia_shp = countries.filterMetadata('COUNTRY', 'equals', 'Colombia')
s5p_metane_dpto = s5p_metane.filterBounds(colombia_shp)
s5p_metane_img = s5p_metane_dpto.map(lambda image: image.clip(colombia_shp))
Map.addLayer(s5p_metane_img, s5p_methane_viz, 'methane in COlombia ')

Map.centerObject(Colombia, 5)


In [12]:
Map.add_colorbar(
    s5p_methane_viz,
    label="Methane meditions (mol/mol)",
    layer_name="Methane colorbar",
    transparent_bg=True
)


In [36]:
import os

In [39]:
out_dir = os.path.join('/home/bojack/Downloads')
html_file = os.path.join(out_dir, 'my_map.html')
Map.to_html(filename=html_file, title='My Map', width='100%', height='880px')

In [40]:
jpg_file = os.path.join(out_dir, 'my_map.jpg')
Map.to_image(filename=jpg_file, monitor=1)